In [57]:
import os
import ROOT 
import pandas as pd
import numpy as np 
import math

In [58]:
l=20
sl=1
a=10
sa=1
t=1
st=0.2

In [59]:
ROOT.gStyle.SetOptFit(1)

data_file_path = os.path.join('data', 'Ozd.csv')

try:
    data = np.loadtxt(data_file_path, delimiter=',',skiprows=1)
    x = data[:, 1]*1e-3
    print(x)
    y = data[:, 0]
    print(y)
    sx = np.full(len(x), 0.01)
    sy = np.full(len(y), 0.1)
    n_points = len(x)
    print(f"Successfully loaded {n_points} points from '{data_file_path}'.")
except (IOError, IndexError) as e:
    print(f"Error: Could not read or parse '{data_file_path}'.")
    exit()


graph = ROOT.TGraphErrors(len(x))
for i in range(len(x)):
    graph.SetPoint(i, x[i], y[i])
    graph.SetPointError(i, sx[i], sy[i])

fit_func = ROOT.TF1("fit_func", "[0] + [1]*x", min(x), max(x))
fit_func.SetParameters(0, 1)  # initial guesses

graph.Fit(fit_func)
r_parameters=[(fit_func.GetParameter(i),fit_func.GetParError(i)) for i in range(2)]

visual_scale_factor =500

print(f"Applying a visual scaling factor of {visual_scale_factor}.")

for i in range(len(x)):
    graph.SetPointError(i, sx[i], sy[i]*3)
graph.SetMarkerStyle(20)
graph.SetMarkerColor(ROOT.kBlue)
graph.SetTitle("Resistance of the Hall Probe ;Current (A);Potential (V)")

c = ROOT.TCanvas("c", "Line Fit", 800, 600)
graph.Draw("AP")
fit_func.Draw("same")
c.Update()
stats = graph.GetListOfFunctions().FindObject("stats")
if stats:
    stats.SetX1NDC(0.15)  # lower-left x (0 to 1)
    stats.SetY1NDC(0.75)  # lower-left y
    stats.SetX2NDC(0.45)  # upper-right x
    stats.SetY2NDC(0.87)   # upper-right y
    stats.SetTextSize(0.03)
    c.Modified()

ROOT.gPad.SetTicks(1, 1)
ROOT.gPad.SetGrid(1, 1)
c.Update()
c.SaveAs("resistivity.pdf")
del c, graph, fit_func


[0.00995 0.0204  0.03025 0.03951 0.0497  0.0583  0.0681  0.0774  0.09
 0.1    ]
[0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.8 5.4]
Successfully loaded 10 points from 'data/Ozd.csv'.
Applying a visual scaling factor of 500.
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =     0.120242
NDf                       =            8
Edm                       =  6.48052e-06
NCalls                    =           97
p0                        =    -0.133191   +/-   0.377935    
p1                        =      54.3176   +/-   6.16417     


Info in <TCanvas::Print>: pdf file resistivity.pdf has been created


### **p1 is resistance of hall probe**

In [60]:
r_parameters

[(-0.1331911453452763, 0.3779348391082072),
 (54.31762687467632, 6.164172480395432)]

from this, resistivity is $$\rho=\frac{R.a}{L}$$

where L = length of probe 20mm   
a=with of the probe (10mm) $\times$ thickness(1+-0.2mm)

In [61]:
r_parameters

[(-0.1331911453452763, 0.3779348391082072),
 (54.31762687467632, 6.164172480395432)]

In [62]:

def resistivity(r):
    return r[1][0]*(t*a)/l
def sigma_r(res,r):
    # --- Calculate squared relative uncertainties ---
    rel_R_sq = math.pow(r[1][1]/ r[1][0], 2)
    rel_w_sq = math.pow(sa / a, 2)
    rel_t_sq = math.pow(st / t, 2)
    rel_L_sq = math.pow(sl / l, 2)

    total_rel_uncertainty = math.sqrt(rel_R_sq + rel_w_sq + rel_t_sq + rel_L_sq)
    return res * total_rel_uncertainty

r=resistivity(r_parameters)*1e-3
sr=sigma_r(r,r_parameters)
print(f"resistivity is {r:.3g}+-{sr:.3g} ohm X meters")

resistivity is 0.0272+-0.00694 ohm X meters


In [63]:
ROOT.gStyle.SetOptFit(1)

data_file_path = os.path.join('data', 'AkimSabit.csv')
try:
    data = np.loadtxt(data_file_path, delimiter=',',skiprows=1)
    print(f"consant I is {float(data[1,0])}mA")
    constI=[float(data[1,0]),0.01]
    x = data[:, 1]
    y = data[:, 2]+76.5
    sx =x*0.05
    sy = np.full(len(y), 0.1)
    n_points = len(x)
    print(f"Successfully loaded {n_points} points from '{data_file_path}'.")
except (IOError, IndexError) as e:
    print(f"Error: Could not read or parse '{data_file_path}'.")
    exit()

graph = ROOT.TGraphErrors(len(x))
for i in range(len(x)):
    graph.SetPoint(i, x[i], y[i])
    graph.SetPointError(i, sx[i], sy[i])

fit_func = ROOT.TF1("fit_func", "[0] + [1]*x", min(x), max(x))
fit_func.SetParameters(0, 0)  # initial guesses

graph.Fit(fit_func)
I_parameters=[(fit_func.GetParameter(i),fit_func.GetParError(i)) for i in range(2)]

visual_scale_factor =100

print(f"Applying a visual scaling factor of {visual_scale_factor}.")

for i in range(len(x)):
    graph.SetPointError(i, sx[i]*2 , sy[i]* visual_scale_factor)
graph.SetMarkerStyle(20)
graph.SetMarkerColor(ROOT.kBlue)
graph.SetTitle(f"Hall Constant with I={constI[0]}mA ;Magnetic Field (mT);Potential (mV)")

c = ROOT.TCanvas("c", "Line Fit", 800, 600)
graph.Draw("AP")
fit_func.Draw("same")
c.Update()
stats = graph.GetListOfFunctions().FindObject("stats")
if stats:
    stats.SetX1NDC(0.55)  # lower-left x (0 to 1)
    stats.SetY1NDC(0.75)  # lower-left y
    stats.SetX2NDC(0.85)  # upper-right x
    stats.SetY2NDC(0.87)   # upper-right y
    stats.SetTextSize(0.03)
    c.Modified()

ROOT.gPad.SetTicks(1, 1)
ROOT.gPad.SetGrid(1, 1)
c.Update()
c.SaveAs("const I.pdf")
del c, graph, fit_func


consant I is 100.1mA
Successfully loaded 10 points from 'data/AkimSabit.csv'.
Applying a visual scaling factor of 100.
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.49855
NDf                       =            8
Edm                       =  2.66229e-07
NCalls                    =          124
p0                        =     0.394373   +/-   0.885527    
p1                        =    -0.721109   +/-   0.0170845   


Info in <TCanvas::Print>: pdf file const I.pdf has been created


In [64]:
I_parameters

[(0.39437340857194814, 0.8855274572043822),
 (-0.7211092181863736, 0.017084533633774556)]

In [65]:
constI

[100.1, 0.01]

### here p1 can be used to work out $R_h$

## $\frac{E_h(mV)}{B(mT)}=p1=R_h\times\frac{I(mA)}{t(mm)}$ 

In [66]:
def hall_coef(r,const):
    hall=r[1][0]*t/const[0]
    rel_R_sq = math.pow(r[1][1]/ r[1][0], 2)
    rel_t_sq = math.pow(st / t, 2)
    rel_C_sq = math.pow(const[1]/const[0], 2)

    total_rel_uncertainty = math.sqrt(rel_R_sq +  rel_t_sq + rel_C_sq)
    return [hall,hall * total_rel_uncertainty]

hall_coefI=hall_coef(I_parameters,constI)
print(f"Hall coef for consant I is {hall_coefI[0]:.3g}+-{-hall_coefI[1]:.3g} m^3/C ")

Hall coef for consant I is -0.0072+-0.00145 m^3/C 


In [67]:
data_file_path = os.path.join('data', 'BSa.csv')

try:
    data = np.loadtxt(data_file_path, delimiter=',',skiprows=1)
    print(f"consant B is {float(data[1,0])}mT")
    constB=[float(data[1,0]),float(data[1,0])*0.05]
    x = data[:, 1]
    y = data[:, 2]
    sx = np.full(len(x), 0.1)
    sy = np.full(len(y), 0.1)
    n_points = len(x)
    print(f"Successfully loaded {n_points} points from '{data_file_path}'.")
except (IOError, IndexError) as e:
    print(f"Error: Could not read or parse '{data_file_path}'.")
    exit()
ROOT.gStyle.SetOptFit(1)


graph = ROOT.TGraphErrors(len(x))
for i in range(len(x)):
    graph.SetPoint(i, x[i], y[i])
    graph.SetPointError(i, sx[i], sy[i])

fit_func = ROOT.TF1("fit_func", "[0] + [1]*x", min(x), max(x))
fit_func.SetParameters(0, 0)  # initial guesses

graph.Fit(fit_func)
B_parameters=[(fit_func.GetParameter(i),fit_func.GetParError(i)) for i in range(2)]

visual_scale_factor =80

print(f"Applying a visual scaling factor of {visual_scale_factor}.")

for i in range(len(x)):
    graph.SetPointError(i, sx[i]*visual_scale_factor , sy[i]* visual_scale_factor)
graph.SetMarkerStyle(20)
graph.SetMarkerColor(ROOT.kBlue)
graph.SetTitle(f"Hall Constant with B={constB[0]}mT;Current (mA);Potential (mV)")

c = ROOT.TCanvas("c", "Line Fit", 800, 600)
graph.Draw("AP")
fit_func.Draw("same")
c.Update()
stats = graph.GetListOfFunctions().FindObject("stats")
if stats:
    stats.SetX1NDC(0.55)  # lower-left x (0 to 1)
    stats.SetY1NDC(0.75)  # lower-left y
    stats.SetX2NDC(0.85)  # upper-right x
    stats.SetY2NDC(0.87)   # upper-right y
    stats.SetTextSize(0.03)
    c.Modified()

ROOT.gPad.SetTicks(1, 1)
ROOT.gPad.SetGrid(1, 1)
c.Update()
c.SaveAs("const B.pdf")
del c, graph, fit_func


consant B is 200.0mT
Successfully loaded 10 points from 'data/BSa.csv'.
Applying a visual scaling factor of 80.
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      44.8795
NDf                       =            8
Edm                       =   4.7889e-07
NCalls                    =           67
p0                        =     0.899733   +/-   0.122986    
p1                        =     -1.48606   +/-   0.0019762   


Info in <TCanvas::Print>: pdf file const B.pdf has been created


In [68]:
B_parameters

[(0.8997332491723463, 0.12298599711984337),
 (-1.4860560670877954, 0.001976202234709174)]

In [69]:
constB*2

[200.0, 10.0, 200.0, 10.0]

In [70]:
hall_coefB=hall_coef(B_parameters,constB)
print(f"Hall coef for constant B is {hall_coefB[0]:.4g}+-{-hall_coefB[1]:.3g} m^3/C ")

Hall coef for constant B is -0.00743+-0.00153 m^3/C 


## we know $\rho\times \mu_h=R_h$

In [71]:
def density(r,sr,hall):
    mu=hall[0]/r
    rel_r_sq = math.pow(sr/ r, 2)
    rel_h_sq = math.pow(hall[1] /hall[0], 2)
    total_rel_uncertainty = math.sqrt(rel_r_sq +  rel_h_sq )
    return [mu,mu * total_rel_uncertainty]
mu1=density(r,sr,hall_coefI)
print(f"Mobility for fixed I is {mu1[0]:.2e}+-{-mu1[1]:.1e}")   
mu2=density(r,sr,hall_coefB)
print(f"Mobility for fixed B is {mu2[0]:.2e}+-{-mu2[1]:.1e}")  
hall_avg=[(hall_coefB[i]+hall_coefI[i])/2 for i in range(2)]
mu3=density(r,sr,hall_avg)
print(f"Mobility for combined result is {mu3[0]:.2e}+-{-mu3[1]:.1e}")   
    

Mobility for fixed I is -2.65e-01+-8.6e-02
Mobility for fixed B is -2.74e-01+-9.0e-02
Mobility for combined result is -2.69e-01+-8.8e-02
